In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

import pandas as pd
import matplotlib.pyplot as plt

In [2]:
torch.cuda.is_available()

True

In [3]:
df = pd.read_csv('../Data/iris.csv')

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [5]:
X = df.drop('target', axis=1).values

In [6]:
y = df['target'].values

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=33)

In [8]:
X_train = torch.FloatTensor(X_train).cuda()
X_test = torch.FloatTensor(X_test).cuda()
y_train = torch.LongTensor(y_train).cuda()
y_test = torch.LongTensor(y_test).cuda()

In [9]:
class Model(nn.Module):
    def __init__(self, in_features=4, h1=8, h2=9, out_features=3):
        super().__init__()
        self.fc1 = nn.Linear(in_features,h1)    # input layer
        self.fc2 = nn.Linear(h1, h2)            # hidden layer
        self.out = nn.Linear(h2, out_features)  # output layer
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.out(x)
        return x

In [10]:
torch.manual_seed(32)
model = Model()

In [11]:
model1 = model.cuda()

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model1.parameters(), lr=0.01)

In [16]:
import time
start = time.time()

epochs = 1000
losses = []

for i in range(epochs):
    i+=1
    y_pred = model1.forward(X_train)
    loss = criterion(y_pred, y_train)
    losses.append(loss)
    
    # a neat trick to save screen space:
    if i%100 == 1:
        print(f'epoch: {i:2}  loss: {loss.item():10.8f}')

    # back propagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


end = time.time() - start
print(f'Total time {end}')

epoch:  1  loss: 0.04125550
epoch: 101  loss: 0.04126381
epoch: 201  loss: 0.04125367
epoch: 301  loss: 0.04125466
epoch: 401  loss: 0.04125239
epoch: 501  loss: 0.04125730
epoch: 601  loss: 0.04125135
epoch: 701  loss: 0.04125137
epoch: 801  loss: 0.04125059
epoch: 901  loss: 0.04125002
Total time 5.694997310638428


In [20]:
next(model1.parameters()).is_cuda

True

In [14]:
correct = 0

with torch.no_grad():
    for i,data in enumerate(X_test):
        y_val = model1.forward(data)
        # remove .argmax().item() to get results tensor
        print(f'{i+1}.) {str(y_val.argmax().item())} {y_test[i]} ')
        
        if y_val.argmax().item() == y_test[i]:
            correct +=1

print(f'We got {correct} correct. ')

1.) 1 1 
2.) 1 1 
3.) 0 0 
4.) 1 1 
5.) 2 2 
6.) 2 2 
7.) 0 0 
8.) 0 0 
9.) 2 2 
10.) 2 2 
11.) 2 2 
12.) 0 0 
13.) 2 2 
14.) 1 1 
15.) 2 2 
16.) 1 1 
17.) 1 2 
18.) 0 0 
19.) 1 1 
20.) 2 2 
21.) 0 0 
22.) 0 0 
23.) 2 2 
24.) 0 0 
25.) 2 2 
26.) 2 2 
27.) 1 1 
28.) 1 1 
29.) 2 2 
30.) 2 2 
We got 29 correct. 
